In [13]:
import pickle
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold

In [14]:
with open("test7", "rb") as f:   # Unpickling
    list_dfs_signif = pickle.load(f)

En esta lista tenemos tuplas con las matrices que hay por cada preproceso, trozo y canal. Estas tuplas tienen la forma:
* DataFrame con la matriz correspondiente
* int que indica el tipo de preproceso
* int que indica el trozo de la grabación
* int que indica el canal

In [15]:
#Listas para guardar listas
lista_prep = []
lista_troz = []
lista_chan = []
# inicializamos los parámetros
prep=0
troz=0
chan=0
# iteramos sobre todas las tuplas
for item in list_dfs_signif:
    # cogemos los parámetros de la nueva tupla
    new_df = item[0].iloc[:,:-1]
    new_prep = item[1]
    new_troz = item[2]
    new_chan = item[3]
    # Comprobamos que el preproceso nuevo y el anterior sean iguales
    if (new_prep == prep):
        # comprobamos que el trozo nuevo y el anterior sean iguales
        if (new_troz == troz):
            # Añadimos el prep, el trozo y el canal el trozo a las columnas:
            df = new_df.add_prefix(str(new_prep)+'_'+str(new_troz)+'_'+str(new_chan)+'_')
            # Guardamos la matriz en la lista correspondiente
            lista_chan.append(df)
            prep = new_prep
            troz = new_troz
            chan = new_chan
        # Si el trozo cambia:
        else:
            # añadimos la lista de canales a la lista de trozos
            lista_troz.append(lista_chan)
            # inicializamos la lista de los canales
            lista_chan = []
            # Añadimos el prep, el trozo y el canal a las columnas:
            df = new_df.add_prefix(str(new_prep)+'_'+str(new_troz)+'_'+str(new_chan)+'_')
            lista_chan.append(df)
            prep = new_prep
            troz = new_troz
            chan = new_chan
    # Si el preproceso cambia:
    else:
        # Añadimos la lista de canales a la lista de trozos
        lista_troz.append(lista_chan)
        # Añadimos la lista de trozos a la lista de preprocesos
        lista_prep.append(lista_troz)
        # Inicializamos la lista de los canales
        lista_chan = []
        # Inicializamos la lista de los trozos
        lista_troz = []
        # Añadimos el prep, el trozo y el canal a las columnas:
        df = new_df.add_prefix(str(new_prep)+'_'+str(new_troz)+'_'+str(new_chan)+'_')
        lista_chan.append(df)
        prep = new_prep
        troz = new_troz
        chan = new_chan
lista_troz.append(lista_chan)
lista_prep.append(lista_troz) 
#list_result.sort(key=lambda x: x[6], reverse=True)

In [16]:
# with open("test11", "wb") as l_prep:   #Pickling
#     pickle.dump(lista_prep, l_prep)

Ahora tengo una lista con 4 listas (cada uno de los preprocesos). Cada una de estas listas tiene a su vez tantas listas como trozos (35) y, cada una de las listas de trozos contiene las matrices DES de todos los canales del trozo.

El siguiente paso es generar nuevas matrices en las listas de trozos, juntando las matrices de canales de 2 en 2, como hicimos con el total de matrices.

In [4]:
def conversor_num_to_chan(num):
    if num == 0:
        a = 'Fp1'
    elif num == 1:
        a = 'Fp2'
    elif num == 2:
        a = 'F3'
    elif num == 3:
        a = 'F4'
    elif num == 4:
        a = 'C3'
    elif num == 5:
        a = 'C4'
    elif num == 6:
        a = 'P3'
    elif num == 7:
        a = 'P4'
    elif num == 8:
        a = 'O1'
    elif num == 9:
        a = 'O2'
    elif num == 10:
        a = 'F7'
    elif num == 11:
        a = 'F8'
    elif num == 12:
        a = 'T7'
    elif num == 13:
        a = 'T8'
    elif num == 14:
        a = 'P7'
    elif num == 15:
        a = 'P8'
    elif num == 16:
        a = 'Fz'
    elif num == 17:
        a = 'Cz'
    elif num == 18:
        a = 'Pz'
    else:
        print('error de canales')
    return a

In [5]:
lista_total = []
lista_new_preproceso = []
for list_trozos in lista_prep:
    lista_new_trozos = []
    for list_channels in list_trozos:
        lista_new_matrices = []
        for i in list(range(len(list_channels))):
            for j in list(range(len(list_channels))):
                if j>i:
                    label = list_dfs_signif[0][0].iloc[:,-1]
                    df1 = list_channels[i]
                    df2 = list_channels[j]
                    # cogemos los números reales de los canales
                    lab_chan_1 = int(df1.columns[0].split('_')[2])
                    lab_chan_2 = int(df2.columns[0].split('_')[2])
                    # los convertimos al nombre del canal
                    lab_chan_1 = conversor_num_to_chan(lab_chan_1)
                    lab_chan_2 = conversor_num_to_chan(lab_chan_2)
                    # Unimos los 2 dfs
                    df3 = pd.concat([df1,df2], axis=1)
                    # Creamos las columnas con los canales y las etiquetas
                    list_lab_chan =  [(lab_chan_1,lab_chan_2)]*df3.shape[0]
                    df3['Label'] = label
                    df3['Channels'] = list_lab_chan
                    df3['chunk'] = [int(df2.columns[0].split('_')[1])]*df3.shape[0]
                    df3['prep'] = [int(df2.columns[0].split('_')[0])]*df3.shape[0]
                    lista_new_matrices.append(df3)
                    lista_total.append(df3)
        lista_new_trozos.append(lista_new_matrices)
    lista_new_preproceso.append(lista_new_trozos)

Estas combinaciones no tienen características con DES:
1. p:1, t:7, c:1
2. p:1, t:11, c:0
3. p:1, t:19, c:15
4. p:1, t:23, c:1
5. p:1, t:24, c:1
6. p:1, t:27, c:1
7. p:1, t:27, c:7
8. p:3, t:34, c:18

In [13]:
from math import factorial
def combinaciones(n,r):
    num = factorial(n)
    den = factorial(r) * factorial(n-r)
    res = num/den
    return int(res)
print(35*combinaciones(19,2)*4)
print(len(lista_total))

print(35*combinaciones(19,2) + \
      29*combinaciones(19,2) + \
       5*combinaciones(18,2) + \
       1*combinaciones(17,2) + \
      35*combinaciones(19,2) + \
      34*combinaciones(19,2) + \
       1*combinaciones(18,2))

23940
23797
23797


Ahora tenemos en la lista `lista_new_preproceso` 4 listas con cada uno de los preprocesos. Dentro de cada una de estas listas están los trozos (35) y dentro de cada lista de trozos están las matrices todas las matrices correspondientes a 2 sensores.

In [6]:
with open("test9", "wb") as lista_nw_preproceso:   #Pickling
    pickle.dump(lista_new_preproceso, lista_nw_preproceso)

In [15]:
list_results = []
# for n_prep, prep in enumerate(lista_new_preproceso):
#     for n_troz, troz in enumerate(prep):
#         for n_chan, mat in enumerate(troz):
for mat in lista_total:
    for eta in [0.01, 0.1, 0.5, 1]:
        for gamma in [0, 1]:
            for max_depth in [2, 3, 6, 12, 24]:
                print('prep:', mat.iloc[0,-1],\
                      'troz:', mat.iloc[0,-2],\
                      'chan:', mat.iloc[0,-3],)
                      #'eta:', eta,'gamma:', gamma,'max_depth:', max_depth)
                # Cogemos los datos
                data_dm = mat.iloc[:,:-4].to_numpy()
                label_dm = mat['Label'].to_numpy()
                dtrain = xgb.DMatrix(data_dm, label=label_dm)
                param = {"max_depth": max_depth, "eta": eta , "gamma": gamma, "objective": "binary:logistic"}
                num_round = 30
                k_folds = StratifiedKFold(n_splits=5)
                res = xgb.cv(
                        param,
                        dtrain,
                        num_round,
                        stratified=True,
                        folds= k_folds,
                        metrics={"error"},
                        seed=0,
                        verbose_eval=0
                    )
                min_row = res[res['test-error-mean'] == res['test-error-mean'].min()].iloc[0,:]
                min_error = round(min_row.iloc[2],4)
                std_error = round(min_row.iloc[3], 4)
                list_results.append((mat.iloc[0,-1], mat.iloc[0,-2], mat.iloc[0,-3], eta, gamma, max_depth, 1-min_error, std_error))
                print('Accuracy máximo obtenido:', 1-min_error, '+-', std_error)


prep: 0 troz: 0 chan: ('Fp1', 'Fp2')
Accuracy máximo obtenido: 0.6937 +- 0.1117
prep: 0 troz: 0 chan: ('Fp1', 'Fp2')
Accuracy máximo obtenido: 0.6443 +- 0.1113
prep: 0 troz: 0 chan: ('Fp1', 'Fp2')
Accuracy máximo obtenido: 0.6276999999999999 +- 0.0963
prep: 0 troz: 0 chan: ('Fp1', 'Fp2')
Accuracy máximo obtenido: 0.6276999999999999 +- 0.0963
prep: 0 troz: 0 chan: ('Fp1', 'Fp2')
Accuracy máximo obtenido: 0.6276999999999999 +- 0.0963
prep: 0 troz: 0 chan: ('Fp1', 'Fp2')
Accuracy máximo obtenido: 0.6937 +- 0.1117
prep: 0 troz: 0 chan: ('Fp1', 'Fp2')
Accuracy máximo obtenido: 0.6443 +- 0.1339
prep: 0 troz: 0 chan: ('Fp1', 'Fp2')
Accuracy máximo obtenido: 0.636 +- 0.1044
prep: 0 troz: 0 chan: ('Fp1', 'Fp2')
Accuracy máximo obtenido: 0.636 +- 0.1044
prep: 0 troz: 0 chan: ('Fp1', 'Fp2')
Accuracy máximo obtenido: 0.636 +- 0.1044
prep: 0 troz: 0 chan: ('Fp1', 'Fp2')
Accuracy máximo obtenido: 0.728 +- 0.1111
prep: 0 troz: 0 chan: ('Fp1', 'Fp2')
Accuracy máximo obtenido: 0.6867 +- 0.0902
prep: 0 

In [ ]:
# Código anterior

# list_results = []
# for n_prep, prep in enumerate(lista_new_preproceso):
#     for n_troz, troz in enumerate(prep):
#         for n_chan, mat in enumerate(troz):
#             for eta in [0.01, 0.1, 0.5, 1]:
#                 for gamma in [0, 1]:
#                     for max_depth in [2, 3, 6, 12, 24]:
#                         print('prep:',n_prep,'troz:', n_troz,'chan:', n_chan,'eta:', eta,'gamma:', gamma,'max_depth:', max_depth)
#                         # Cogemos los datos
#                         data_dm = mat.iloc[:,:-1].to_numpy()
#                         label_dm = mat['Label'].to_numpy()
#                         dtrain = xgb.DMatrix(data_dm, label=label_dm)
#                         param = {"max_depth": max_depth, "eta": eta , "gamma": gamma, "objective": "binary:logistic"}
#                         num_round = 30
#                         k_folds = StratifiedKFold(n_splits=5)
#                         res = xgb.cv(
#                                 param,
#                                 dtrain,
#                                 num_round,
#                                 stratified=True,
#                                 folds= k_folds,
#                                 metrics={"error"},
#                                 seed=0,
#                                 verbose_eval=0
#                             )
#                         min_row = res[res['test-error-mean'] == res['test-error-mean'].min()].iloc[0,:]
#                         min_error = round(min_row.iloc[2],4)
#                         std_error = round(min_row.iloc[3], 4)
#                         list_results.append((n_prep, n_troz, n_chan, eta, gamma, max_depth, 1-min_error, std_error))
#                         print('Accuracy máximo obtenido:', 1-min_error, '+-', std_error)



In [ ]:
#list_acc =[]
#for (prep, troz, chan, eta, gamma, max_depth, acc, std_error) in list_results:
#    list_acc.append(acc)

In [16]:
list_results.sort(key=lambda x: x[6], reverse=True)

In [17]:
df_results_xgb = pd.DataFrame(list_results, columns = ['preproceso', 'trozo', 'canal', 'eta', 'gamma',
                                                      'max_depth', 'accuracy', 'std_error'])


In [18]:
# Actual recortada
df_results_xgb.head(50)

,preproceso,trozo,canal,eta,gamma,max_depth,accuracy,std_error
0,0,0,"(F3, T7)",0.10,1,3,0.8187,0.0658
1,0,0,"(F3, T7)",0.10,0,3,0.8100,0.0620
2,0,0,"(F3, T7)",0.50,0,3,0.8023,0.0826
3,0,0,"(F3, T7)",0.10,1,6,0.8020,0.0597
4,0,0,"(F3, T7)",0.10,1,12,0.8020,0.0597
5,0,0,"(F3, T7)",0.10,1,24,0.8020,0.0597
6,0,0,"(O1, T7)",0.50,1,6,0.7943,0.1023
7,0,0,"(O1, T7)",0.50,1,12,0.7943,0.1023
8,0,0,"(O1, T7)",0.50,1,24,0.7943,0.1023
9,0,0,"(F3, T7)",0.10,0,6,0.7940,0.0853


In [34]:
# Antiguo
df_results_xgb.head(50)

,preproceso,trozo,canal,eta,gamma,max_depth,accuracy,std_error
0,0,31,"(P7, Cz)",0.01,0,2,0.8683,0.0461
1,0,31,"(P7, Cz)",0.01,1,2,0.8683,0.0461
2,0,29,"(Fp2, P7)",0.10,0,2,0.8593,0.0429
3,0,29,"(Fp2, P7)",0.10,1,2,0.8593,0.0429
4,2,0,"(C3, P7)",0.10,1,3,0.8527,0.1156
5,2,8,"(P7, Cz)",0.50,0,3,0.8520,0.0915
6,0,31,"(P3, P7)",1.00,0,3,0.8517,0.0609
7,0,31,"(P7, Cz)",0.10,0,2,0.8517,0.0549
8,0,31,"(P7, Cz)",0.10,1,2,0.8517,0.0549
9,0,31,"(P3, P7)",0.10,0,2,0.8513,0.0421


In [33]:
with open("prueba1", "wb") as f:   #Pickling
    pickle.dump(df_results_xgb, f)

with open("prueba2", "wb") as p:   #Pickling
    pickle.dump(list_results, p)

In [5]:
with open("prueba1", "rb") as f:   # Unpickling
    df_results_xgb = pickle.load(f) 

In [7]:
df_results_xgb.head(20)

,preproceso,trozo,canal,eta,gamma,max_depth,accuracy,std_error
0,0,31,"(P7, Cz)",0.01,0,2,0.8683,0.0461
1,0,31,"(P7, Cz)",0.01,1,2,0.8683,0.0461
2,0,29,"(Fp2, P7)",0.10,0,2,0.8593,0.0429
3,0,29,"(Fp2, P7)",0.10,1,2,0.8593,0.0429
4,2,0,"(C3, P7)",0.10,1,3,0.8527,0.1156
5,2,8,"(P7, Cz)",0.50,0,3,0.8520,0.0915
6,0,31,"(P3, P7)",1.00,0,3,0.8517,0.0609
7,0,31,"(P7, Cz)",0.10,0,2,0.8517,0.0549
8,0,31,"(P7, Cz)",0.10,1,2,0.8517,0.0549
9,0,31,"(P3, P7)",0.10,0,2,0.8513,0.0421
